### This script accepts tiff files from microscopy time-lapse recordings and restructures channel and z-plane order to prepare the files for analysis

In [ ]:
import numpy as np, os, tifffile
from glob import glob

### function definitions

In [ ]:
# Function to deinterleave a TIFF file containing stacked composite images
def deinterleave_TZC(input_tif: str, output_tif: str, n_channels: int, n_zplanes: int):
    """
    Input TIFF is a sequence of 2D grayscale images like:
    Page 0: frame 0, z-pos 0, channel 0 (R)
    Page 1: frame 0, z-pos 1, channel 0 (R)
    Page 2: frame 0, z-pos 2, channel 0 (R)
    Page 3: frame 1, z-pos 0, channel 1 (G)
    Page 4: frame 1, z-pos 1, channel 1 (G)
    Page 5: frame 1, z-pos 2, channel 1 (G)
    Page 6: frame 2, z-pos 0, channel 2 (B)
    Page 7: frame 2, z-pos 1, channel 2 (B)
    Page 8: frame 2, z-pos 2, channel 2 (B)
    ...

    Output TIFF is a sequence of stacked composite images
    """
    
    # Read TIFF file
    reader = tifffile.TiffFile(input_tif)
    n_pages = len(reader.pages)  # Total number of pages in the TIFF file

    frame = reader.pages[0].asarray()  # Read the first frame to get dimensions
    height, width = frame.shape  # Get height and width of the frames

    # Check if the number of pages is divisible by (number of channels * number of z-planes)
    assert n_pages % (n_channels * n_zplanes) == 0
    n_timepoints = n_pages // (n_channels * n_zplanes)  # Calculate the number of timepoints
    print(f"number of timepoints: {n_timepoints} | number of pages: {n_pages} | frame size: {frame.shape}")

    # Initialize an empty array to store deinterleaved data
    out = np.empty((n_timepoints, n_zplanes, n_channels, height, width), dtype=np.uint16)

    # Loop through each timepoint, channel, and z-plane to deinterleave the data
    for t in range(n_timepoints):
        for c in range(n_channels):
            for z in range(n_zplanes):
                page_idx = (t * n_channels + c) * n_zplanes + z
                out[t, z, c, :, :] = reader.pages[page_idx].asarray()  # Store the data
        
    # Write the deinterleaved data to a TIFF file
    tifffile.imwrite(output_tif, out, imagej=True, metadata={'mode': 'composite'})


# Function to trim a TIFF file to a specified frame
def trim_tiff(input_tif: str, output_tif: str, last_frame: int):
    # Read TIFF file
    img = tifffile.imread(input_tif)
    img = img[:last_frame, :, :]  # Trim the TIFF file to the specified frame
    tifffile.imsave(output_tif, img)


# Function to trim all TIFF files in a directory
def trim_all_tifs_in_directory(indir: str, outdir: str, lastFrame: int):
    # Get a list of TIFF files in the input directory
    targets = sorted(glob(os.path.join(indir, "*.tif")))
    # Check if the output directory already exists
    if os.path.isdir(outdir):
        raise OSError(f"output directory {outdir} must not already exist")
    os.mkdir(outdir)  # Create the output directory
    # Loop through each TIFF file, trim it, and save the trimmed version in the output directory
    for t in targets:
        print(f"working on {t}")
        o = os.path.join(outdir, os.path.basename(t))
        assert not os.path.isfile(o)
        trim_tiff(t, o, lastFrame)


# Function to deinterleave all TIFF files in a directory
def deinterleave_all_tifs_in_directory(indir: str, outdir: str, n_chan: int, n_zpl: int):
    # Get a list of TIFF files in the input directory
    targets = sorted(glob(os.path.join(indir, "*.tif")))
    # Check if the output directory already exists
    if os.path.isdir(outdir):
        raise OSError(f"output directory {outdir} must not already exist")
    os.mkdir(outdir)  # Create the output directory
    # Loop through each TIFF file, deinterleave it, and save the deinterleaved version in the output directory
    for t in targets:
        print(f"working on {t}")
        o = os.path.join(outdir, os.path.basename(t))
        assert not os.path.isfile(o)
        deinterleave_TZC(t, o, n_chan, n_zpl)


### Define working folder
Which experiment and recording session do you want to deinterleave?

In [1]:
exp_name     = "Experiment_Title"
session_name = "ImagingSession_01_1"

### Trim files if needed
empty or unusable frames that were caught during image aquisition can be cropped out here

In [ ]:
target_dir = "/mnt/ELNID_Data/ELN/ImagingData/" + exp_name + "/" + session_name
output_dir = "/mnt/ELNID_Data/ELN/ImagingData/" + exp_name + "/" + session_name + "_trimmed"

trim_all_tifs_in_directory(target_dir, output_dir, lastFrame = 87)

### DeInterleave all files

In [ ]:
target_dir = "/mnt/ELNID_Data/ELN/ImagingData/" + exp_name + "/" + session_name
output_dir = "/mnt/ELNID_Data/ELN/ImagingData/" + exp_name + "/deInterleaved/" + session_name

deinterleave_all_tifs_in_directory(target_dir, output_dir, n_chan = 3, n_zpl = 1)
